In [5]:
import pandas as pd
from datetime import datetime
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [50]:
data = pd.read_csv("SOLUSDT-15m-180801.csv")

rate_of_change_1 = ((data["close"] - data["open"]) / data["open"]).to_numpy()[:-127]  # 15 Minutes
rate_of_change_4 = ((data["close"].shift(-3) - data["open"]) / data["open"]).to_numpy()[:-127]  # 1 Hour
rate_of_change_32 = ((data["close"].shift(-31) - data["open"]) / data["open"]).to_numpy()[:-127]  # 8 Hours
rate_of_change_128 = ((data["close"].shift(-127) - data["open"]) / data["open"]).to_numpy()[:-127]  # 1 Day

X1 = rate_of_change_1[:-128]
Y1 = rate_of_change_1[128:]
X2 = rate_of_change_4[:-128]
Y2 = rate_of_change_4[128:]
X3 = rate_of_change_32[:-128]
Y3 = rate_of_change_32[128:]
X4 = rate_of_change_128[:-128]
Y4 = rate_of_change_128[128:]

X = np.vstack((X1, X2, X3, X4)).T
Y = np.vstack((Y1, Y2, Y3, Y4)).T

X.shape, Y.shape

((98206, 4), (98206, 4))

In [51]:
INPUT_WINDOW = 100

In [52]:
import torch
from torch import nn
import torch.functional as F
from torch.utils.data import TensorDataset, DataLoader

X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

Xtrain = X[:int(len(X) * 0.8)]
Ytrain = Y[:int(len(Y) * 0.8)]
Xtest = X[int(len(X) * 0.8):]
Ytest = Y[int(len(Y) * 0.8):]

dataset = TensorDataset(Xtrain, Ytrain)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


class TradingNN(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(TradingNN, self).__init__()
        self.lin1 = nn.Linear(input_dim, 64)
        self.lin2 = nn.Linear(64, 32)
        self.lin3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = torch.relu(self.lin1(x))
        x = torch.dropout(x, 0.8, train=self.training)
        x = torch.relu(self.lin2(x))
        x = torch.dropout(x, 0.8, train=self.training)
        x = self.lin3(x)
        return x

In [53]:
def semantic_loss(y_pred, y_true):
    return torch.mean(torch.abs(y_pred - y_true), dim=0)

In [54]:
model = TradingNN(4, 4)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 1000

for epoch in range(EPOCHS):
    for X_batch, Y_batch in tqdm(dataloader):
        optimizer.zero_grad()
        Y_pred = model(X_batch)
        loss = criterion(Y_pred, Y_batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch} Loss: {loss.item()}")
    
    if epoch % 10 == 0:
        with torch.no_grad():
            model.eval()
            Y_pred = model(Xtest)
            loss = criterion(Y_pred, Ytest)
            print(f"Test Loss: {loss.item()}")
            print(f"Semantic loss {semantic_loss(Y_pred, Ytest)}")
            model.train()

100%|██████████| 1228/1228 [00:01<00:00, 824.58it/s]


Epoch 0 Loss: 0.0012931316159665585
Test Loss: 0.0009222638327628374
Semantic loss tensor([0.0036, 0.0070, 0.0193, 0.0406])


100%|██████████| 1228/1228 [00:01<00:00, 738.69it/s]


Epoch 1 Loss: 0.0008640593150630593


100%|██████████| 1228/1228 [00:01<00:00, 792.25it/s]


...

100%|██████████| 1228/1228 [00:01<00:00, 757.82it/s]


Epoch 170 Loss: 0.0009446237236261368
Test Loss: 0.0009206424583680928
Semantic loss tensor([0.0036, 0.0070, 0.0193, 0.0405])


100%|██████████| 1228/1228 [00:01<00:00, 757.03it/s]


Epoch 171 Loss: 0.0009418692789040506


 31%|███       | 379/1228 [00:00<00:01, 647.59it/s]